Sentiment Analysis for Restaurant Review dataset using Naive Bayes Word Embedding-TF-IDF Vectorizer ,Count Vectorizer
Glove with Logistic Regression

In [22]:
!pip install tensorflow

In [37]:
!pip install torch torchtext


In [2]:
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [3]:
# Download the necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Load the dataset
df = pd.read_csv('/content/Restaurant_Reviews.csv')  # Update with your dataset file name and path
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
# Preprocess the data
stop_words = set(stopwords.words('english'))

def preprocess(text):
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha()]
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['Review'] = df['Review'].apply(preprocess)

In [6]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

#TF-IDF Tokenizer

In [13]:
# Extract features from the reviews using TF-IDF vectorization
vectorizer = TfidfVectorizer(tokenizer=word_tokenize, lowercase=False)
train_features_tfidf = vectorizer.fit_transform(train_data['Review'])
test_features_tfidf = vectorizer.transform(test_data['Review'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [14]:
# Train the Naive Bayes classifier (TF-IDF)
classifier = MultinomialNB()
classifier.fit(train_features_tfidf, train_data['Liked'])

# Make predictions on the test set
predictions_tfidf = classifier.predict(test_features_tfidf)

In [15]:
# Calculate evaluation metrics
true_labels = test_data['Liked']
precision = precision_score(true_labels, predictions_tfidf, average='macro')
recall = recall_score(true_labels, predictions_tfidf, average='macro')
f1 = f1_score(true_labels, predictions_tfidf, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 0.7748274827482748
Recall: 0.7752403846153846
F1 Score: 0.774859287054409


#Using Count Vectorizer

In [17]:
# Extract features from the reviews using Bag-of-Words vectorization
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer=word_tokenize, lowercase=False)
train_features_count = vectorizer.fit_transform(train_data['Review'])
test_features_count = vectorizer.transform(test_data['Review'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [20]:
# Train the Naive Bayes classifier (CountF)
classifier = MultinomialNB()
classifier.fit(train_features_count, train_data['Liked'])

# Make predictions on the test set
predictions_count = classifier.predict(test_features_count)

In [21]:
# Calculate evaluation metrics
true_labels = test_data['Liked']
precision = precision_score(true_labels, predictions_count, average='macro')
recall = recall_score(true_labels, predictions_count, average='macro')
f1 = f1_score(true_labels, predictions_tfidf, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 0.7552255225522553
Recall: 0.7556089743589745
F1 Score: 0.7549448625940838


#Word2vec Glove Embedding with Logistic Regression Classifier

In [24]:
# Download and load GloVe word embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

glove_file = 'glove.6B.300d.txt'

--2023-07-07 14:43:57--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-07-07 14:43:57--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-07-07 14:43:57--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [38]:
import torch
import torchtext

glove_file = '/content/glove.6B.300d.txt'  # Update with the correct path to your unzipped GloVe file

# Load GloVe embeddings
glove = torchtext.vocab.GloVe(name='6B', dim=300, cache='.vector_cache')

# Access word vectors
word_vectors = glove.vectors


100%|█████████▉| 399999/400000 [01:06<00:00, 6012.17it/s]


In [42]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['Review'], df['Liked'], test_size=0.2, random_state=42)

In [54]:
from torch.nn.utils.rnn import pad_sequence
from sklearn.linear_model import LogisticRegression
X_train_vectors = []
X_test_vectors = []
max_sequence_length = 100  # Maximum length for padding or truncation

for sentence in X_train:
    vectors = []
    for word in sentence.split():
        if word in glove.stoi:
            vectors.append(word_vectors[glove.stoi[word]])
    if len(vectors) > max_sequence_length:
        vectors = vectors[:max_sequence_length]  # Truncate if sequence length exceeds max length
    elif len(vectors) < max_sequence_length:
        vectors.extend([torch.zeros_like(word_vectors[0])] * (max_sequence_length - len(vectors)))  # Pad if sequence length is less than max length
    X_train_vectors.append(torch.cat(vectors))

for sentence in X_test:
    vectors = []
    for word in sentence.split():
        if word in glove.stoi:
            vectors.append(word_vectors[glove.stoi[word]])
    if len(vectors) > max_sequence_length:
        vectors = vectors[:max_sequence_length]  # Truncate if sequence length exceeds max length
    elif len(vectors) < max_sequence_length:
        vectors.extend([torch.zeros_like(word_vectors[0])] * (max_sequence_length - len(vectors)))  # Pad if sequence length is less than max length
    X_test_vectors.append(torch.cat(vectors))

# Convert the vectors to numpy arrays
X_train_vectors = np.stack([vec.numpy() for vec in X_train_vectors])
X_test_vectors = np.stack([vec.numpy() for vec in X_test_vectors])

# Train the Logistic Regression classifier
classifier = LogisticRegression()
classifier.fit(X_train_vectors, y_train)

# Make predictions on the test set
predictions = classifier.predict(X_test_vectors)

# Calculate evaluation metrics
precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precision: 0.7375640639131745
Recall: 0.7367788461538461
F1 Score: 0.7349403615813558
